In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
from mpl_toolkits.mplot3d import Axes3D
import os
from matplotlib.colors import LinearSegmentedColormap
import future.utils
import json
import numpy as np
import tqdm
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
sns.set_style("whitegrid")

In [20]:
print('matplotlib: {}'. format(mpl.__version__))

matplotlib: 3.1.0


In [21]:
def hex_to_rgb(value):
    '''
    Converts hex to rgb colours
    value: string of 6 characters representing a hex colour.
    Returns: list length 3 of RGB values'''
    value = value.strip("#") # removes hash symbol if present
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_dec(value):
    '''
    Converts rgb to decimal colours (i.e. divides each value by 256)
    value: list (length 3) of RGB values
    Returns: list (length 3) of decimal values'''
    return [v/256 for v in value]

def get_continuous_cmap(hex_list, float_list=None):
    ''' creates and returns a color map that can be used in heat map figures.
        If float_list is not provided, colour map graduates linearly between each color in hex_list.
        If float_list is provided, each color in hex_list is mapped to the respective location in float_list. 
        
        Parameters
        ----------
        hex_list: list of hex code strings
        float_list: list of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        
        Returns
        ----------
        colour map'''
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
    return cmp

heatmap_hex_list = ['#357db0', '#ce2626']
heatmap_cmap=get_continuous_cmap(heatmap_hex_list)

spaghetti_hex_list = ['#357db0', '#18A558', '#ce2626']
spaghetti_cmap=get_continuous_cmap(spaghetti_hex_list)



In [22]:
def from_res_to_iterations(name, nr):
    res = open("{}.csv".format(name))
    iterations = []
    it = 0
    for line in res.readlines():
        iterations.append(dict())
        iterations[it]['iteration'] = it
        iterations[it]['status'] = dict()

        opinions = line.strip().split(',')

        for i in range(len(opinions)):
            iterations[it]['status'][i] = float(opinions[i])
            iterations[it]['node_count'] = {0: 0},
            iterations[it]['status_delta'] = {0: 0}

        it += 1
        
    return iterations

def evolution(iterations, hex_list, fig, ax):

    spaghetti_hex_list = ['#357db0', '#18A558', '#ce2626']


    """
    Generates the plot

    :param filename: Output filename
    :param percentile: The percentile for the trend variance area
    """

    nodes2opinions = {}
    node2col = {}

    last_it = iterations[-1]['iteration'] + 1
    last_seen = {}

    for it in iterations:
        sts = it['status']
        its = it['iteration']
        for n, v in sts.items():
            if n in nodes2opinions:
                last_id = last_seen[n]
                last_value = nodes2opinions[n][last_id]

                for i in range(last_id, its):
                    nodes2opinions[n][i] = last_value

                nodes2opinions[n][its] = v
                last_seen[n] = its
            else:
                nodes2opinions[n] = [0]*last_it
                nodes2opinions[n][its] = v
                last_seen[n] = 0
                if v < 0.33:
                    node2col[n] = spaghetti_hex_list[0]
                elif 0.33 <= v <= 0.66:
                    node2col[n] = spaghetti_hex_list[1]
                else:
                    node2col[n] = spaghetti_hex_list[2]

    mx = 0
    for k, l in future.utils.iteritems(nodes2opinions):
        if mx < last_seen[k]:
            mx = last_seen[k]
        x = list(range(0, last_seen[k]))
        y = l[0:last_seen[k]]
        ax.plot(x, y, lw=1.5, alpha=0.5, color=node2col[k])
    
    # ax.set_ylim(-0.1, 1.1)
    # ax.set_xlim(0, 1000)
    # ax.tick_params(axis='both', which='major', labelsize=4, pad=0) 
    # ax.set_xlabel("o", fontsize=5)
    # ax.set_ylabel("t", fontsize=5)               
    # plt.grid(axis = 'both', which='both')
    plt.tight_layout()

    
from tqdm import tqdm

def spaghettigridbye(results, imgfolder):
    mops = list(results.media_op.unique())
    pmlist = sorted(list(results.p_media.unique()))
    elist = sorted(list(results.eps.unique()))
    glist=sorted(list(results.gam.unique()))
    sns.set_style("whitegrid")
    i=0
    if imgfolder == "moderate":
        mo = '0.5'
    for e in elist:
        fig, axes = plt.subplots(ncols=len(glist), nrows=len(pmlist), figsize=(6, 6), dpi=600, sharey=True)
        row=0
        for pm in pmlist:
            with tqdm(total=len(pmlist)*len(glist)) as pbar:
                col=0
                for g in glist:
                    name = f'for_spaghetti media mo[{mo}] p{pm} e{e} g{g} gm{g} mi1000000'
                    results = results[results['media_op']==mo]
                    results = results[results['eps']==e]
                    results = results[results['p_media']==pm]
                    results = results[results['gam']==g]
                    iterations = from_res_to_iterations(name, 1)
                    evolution(iterations, spaghetti_hex_list, fig=fig, ax=axes[row,col])
                    axes[row,col].set_title(r"$p_m$={}, $\epsilon$={}, $\gamma={}$".format(pm, e, g), fontsize=5)
                    col+=1    
            row+=1
            pbar.update(1)
        name = f'spaghetti media {imgfolder} e{e}'
        i+=1
        plt.savefig(f"plots/{imgfolder}/{name}.png")
        plt.close()

# Preliminaries

In [23]:
results = pd.read_csv("aggregate/results_media_model.csv", index_col=[0])

In [24]:
results = results[results['gam'] != 0.25]
results

,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,...,median_opinion,avg_00_perc,avg_05_perc,avg_095_perc,avg_005_perc,10B_avg_entr,10B_std_entr,100B_avg_entr,100B_std_entr,100B_norm_entr
0,100,1.0,0.3,0.00,0.00,0.5,1000000,0.05;0.95,1.917247,0.139437,...,0.547112,0.4966,0.0048,0.4986,0.4966,0.682894,0.059351,0.683276,0.060095,0.167361
1,100,1.0,0.2,0.75,0.75,0.5,1000000,0.05;0.5;0.95,2.861434,0.253250,...,0.500045,0.2843,0.4315,0.2680,0.2843,1.182315,0.141936,1.186385,0.145965,0.290592
2,100,1.0,0.3,0.00,0.00,0.2,1000000,0.05;0.95,1.771977,0.290598,...,0.512833,0.4441,0.1297,0.4175,0.4441,0.593931,0.165382,0.594069,0.165018,0.145511
3,100,1.0,0.4,1.25,1.25,0.2,1000000,0.5,1.060078,0.065546,...,0.500177,0.0178,0.9721,0.0101,0.0178,0.266965,0.256676,0.281470,0.264232,0.068943
4,100,1.0,0.5,0.00,0.00,0.4,1000000,0.5,1.000000,0.000000,...,0.500133,0.0000,1.0000,0.0000,0.0000,0.024558,0.106782,0.024558,0.106782,0.006015
5,100,1.0,0.4,1.25,1.25,0.4,1000000,0.5,1.082641,0.062735,...,0.500228,0.0204,0.9617,0.0179,0.0204,0.323349,0.266925,0.341282,0.270358,0.083593
6,100,1.0,0.2,1.00,1.00,0.2,1000000,0.5,3.576107,0.867551,...,0.501146,0.0437,0.4579,0.1013,0.1016,1.440886,0.198281,1.930834,0.306728,0.472937
7,100,1.0,0.4,0.50,0.50,0.5,1000000,0.05;0.5;0.95,9.099572,9.236940,...,0.509467,0.0096,0.5899,0.0310,0.0250,0.629459,0.640735,1.485249,1.600427,0.363795
8,100,1.0,0.3,1.25,1.25,0.1,1000000,0.0,2.121877,1.243069,...,0.520626,0.3060,0.2703,0.0157,0.3060,0.732725,0.283451,1.114823,0.486162,0.273064
9,100,1.0,0.5,1.00,1.00,0.1,1000000,0.05;0.5;0.95,1.758971,0.712546,...,0.494526,0.0000,0.9916,0.0000,0.0000,0.258974,0.226430,0.756872,0.467889,0.185388


In [25]:
gams = list(results.gam.unique())
p_medias = list(results.p_media.unique())
epss = list(results.eps.unique())
mos = list(results.media_op.unique())
print(gams, p_medias, epss, mos)

[0.0, 0.75, 1.25, 1.0, 0.5, 1.5] [0.5, 0.2, 0.4, 0.1, 0.3, 0.0] [0.3, 0.2, 0.4, 0.5] ['0.05;0.95', '0.05;0.5;0.95', '0.5', '0.0']


In [26]:
balanced = results[results['media_op'] == '0.05;0.5;0.95']
print(len(balanced))
gams = list(balanced.gam.unique())
p_medias = list(balanced.p_media.unique())
epss = list(balanced.eps.unique())
mos = list(balanced.media_op.unique())
print(gams, p_medias, epss, mos)

144
[0.75, 0.5, 1.0, 1.5, 1.25, 0.0] [0.5, 0.1, 0.4, 0.0, 0.2, 0.3] [0.2, 0.4, 0.5, 0.3] ['0.05;0.5;0.95']


In [27]:
polarised = results[results['media_op'] == '0.05;0.95']
print(len(polarised))
gams = list(polarised.gam.unique())
p_medias = list(polarised.p_media.unique())
epss = list(polarised.eps.unique())
mos = list(polarised.media_op.unique())
print(gams, p_medias, epss, mos)

144
[0.0, 1.5, 0.5, 1.0, 1.25, 0.75] [0.5, 0.2, 0.4, 0.3, 0.0, 0.1] [0.3, 0.4, 0.2, 0.5] ['0.05;0.95']


In [28]:
extremist = results[results['media_op'] == '0.0']
print(len(extremist))
gams = list(extremist.gam.unique())
p_medias = list(extremist.p_media.unique())
epss = list(extremist.eps.unique())
mos = list(extremist.media_op.unique())
print(gams, p_medias, epss, mos)

144
[1.25, 1.0, 0.0, 0.5, 0.75, 1.5] [0.1, 0.3, 0.0, 0.2, 0.5, 0.4] [0.3, 0.5, 0.2, 0.4] ['0.0']


In [29]:
moderate = results[results['media_op'] == '0.5']
print(len(moderate))
gams = list(moderate.gam.unique())
p_medias = list(moderate.p_media.unique())
epss = list(moderate.eps.unique())
mos = list(moderate.media_op.unique())
print(gams, p_medias, epss, mos)

144
[1.25, 0.0, 1.0, 0.75, 1.5, 0.5] [0.2, 0.4, 0.5, 0.0, 0.3, 0.1] [0.4, 0.5, 0.2, 0.3] ['0.5']


In [30]:
deffuant = results[results['gam']==0.0]

In [31]:
measures = [
 'avg_pwdist',
 'std_pwdist',
 'avg_niter',
 'std_niter',
 'avg_ncluster_0.001',
 'std_ncluster_0.001',
 'avg_opinion',
 'std_opinions',
 'avg_05_perc',
 '100B_avg_entr',
 '100B_std_entr',
 '100B_norm_entr']

In [32]:
print(measures)

['avg_pwdist', 'std_pwdist', 'avg_niter', 'std_niter', 'avg_ncluster_0.001', 'std_ncluster_0.001', 'avg_opinion', 'std_opinions', 'avg_05_perc', '100B_avg_entr', '100B_std_entr', '100B_norm_entr']


# Bounded confidence model with media

### Heatmap

In [33]:
sns.set(font_scale=1.6)
for measure in measures:
    i=0
    for mo in ['0.5', '0.0', '0.05;0.95', '0.05;0.5;0.95']:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5), dpi=600)
        titles = ['moderate', 'extremist', 'polarized', 'balanced']
        x = "p_media"
        y = "eps"
        data = deffuant[deffuant["media_op"]==mo]
        df = data.pivot(x, y, measure)
        a=sns.heatmap(df, cbar=True, fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap)
        ax.set_xlabel(r'$\epsilon$', fontsize=30)
        ax.set_ylabel(r'$p_m$', fontsize=30)
        plt.savefig(f'plots/deffuant/{measure}/{measure}_{titles[i]}_deffuant_heatmap.png')
        plt.close()
        i+=1


### Surface plots

In [35]:
sns.set(font_scale=1.8)
sns.set_style("whitegrid")
for measure in measures:
    i=0
    for mo in ['0.5', '0.0', '0.05;0.95', '0.05;0.5;0.95']:
        titles = ['moderate', 'extremist', 'polarized', 'balanced']
        x = "p_media"
        y = "eps"
        data = deffuant[deffuant["media_op"]==mo]
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(data['eps'], data['p_media'], data[measure], vmin=min(deffuant[measure]), vmax=max(deffuant[measure]), cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        surf=ax.plot_trisurf(data['eps'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        # fig.colorbar(surf, shrink=0.5, aspect=6).ax.tick_params(labelsize=22)
        ax.set_xlabel(r'$\epsilon$', fontsize=40, labelpad=30)
        ax.set_ylabel(r'$p_m$', fontsize=50, labelpad=30)
        ax.tick_params(axis='x', which='major', labelsize=22, pad=2, labelrotation=45)
        ax.tick_params(axis='y', which='major', labelsize=22, pad=10)
        ax.tick_params(axis='z', which='major', labelsize=22, pad=10)  
        ax.set_zlim(0.0, 1.0) 
        plt.savefig(f'plots/deffuant/{measure}/{measure}_{titles[i]}_deffuant_surface.png')
        plt.close() 
        i+=1

### Spaghetti

In [50]:
import sys
sys.path.append("/../Lib/")
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")

n = 100
graph = nx.complete_graph(n)
max_it = 1000000
for mo in [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]:
    for pm in [0.0, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0]:
                if not os.path.exists(f"plots/deffuant/spaghetti/spaghetti_mo{mo}_pm{pm}_e{e}_g{g}.png"):
                    print(f"doing spaghetti_mo{mo}_pm{pm}_e{e}_g{g}")
                    model = op.AlgorithmicBiasMediaModel(graph)
                    config = mc.Configuration()
                    config.add_model_parameter("epsilon", e)
                    config.add_model_parameter("gamma", g)
                    config.add_model_parameter("gamma_media", g)
                    config.add_model_parameter("k", len(mo))
                    config.add_model_parameter("p", pm)
                    model.set_initial_status(config)
                    model.set_media_opinions(mo)
                    # # Simulation execution
                    iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                    fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                    evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                    plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                    plt.savefig(f"plots/deffuant/spaghetti_mo{mo}_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                    plt.close()
                else:
                    print(f"plot plots/deffuant/spaghetti_mo{mo}_pm{pm}_e{e}_g{g}.png already present")
                    continue

  0%|                                                                          | 19/1000000 [00:00<1:29:13, 186.77it/s]

plot plots/deffuant/spaghetti_mo[0.0]_pm0.0_e0.2_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.0_e0.3_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.0_e0.4_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.0_e0.5_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.5_e0.2_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.5_e0.3_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.5_e0.4_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.0]_pm0.5_e0.5_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_pm0.0_e0.2_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_pm0.0_e0.3_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_pm0.0_e0.4_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_pm0.0_e0.5_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_pm0.5_e0.2_g0.0.png already present
plot plots/deffuant/spaghetti_mo[0.5]_

  0%|                                                                          | 22/1000000 [00:00<1:19:02, 210.87it/s]

doing spaghetti_mo[0.05, 0.5, 0.95]_pm0.5_e0.5_g0.0


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [1:28:54<00:00, 187.45it/s]


OverflowError: Exceeded cell block limit (set 'agg.path.chunksize' rcparam)

OverflowError: Exceeded cell block limit (set 'agg.path.chunksize' rcparam)

<Figure size 4200x3000 with 1 Axes>

# A single - moderate - media

### Heatmap

In [38]:
sns.set(font_scale=1.0)
for measure in measures:
    for eps in sorted(moderate.eps.unique()):
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = moderate[moderate["eps"]==eps]
        df = data.pivot(x, y, measure)
        if measure == "avg_niter":
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".0f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(moderate[measure]), vmax=max(moderate[measure]))
        else:
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(moderate[measure]), vmax=max(moderate[measure]))
        ax.set_xlabel(r'$\gamma$', fontsize=15)
        ax.set_ylabel('', fontsize=15)
        ax.set_title(r"$\epsilon$={}", fontsize=15, pad = 60)
        ax.set_ylabel(r'$p_m$', fontsize=15)
        plt.savefig(f"plots/moderate/{measure}/moderate_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()

        # relativa colorbar 
        fig, ax = plt.subplots(figsize=(7, 1), dpi=600)
        fig.subplots_adjust(bottom=0.5)

        cmap = heatmap_cmap
        norm = mpl.colors.Normalize(vmin=min(moderate[measure]), vmax=max(moderate[measure]))

        cb = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=heatmap_cmap),
                    cax=ax, orientation='horizontal')
        cb.outline.set_visible(False)
        plt.savefig(f"plots/moderate/colorbar_moderate_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()

### surface

In [39]:
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
for measure in measures:
    for eps in sorted(moderate.eps.unique()):
        x = "gam"
        y = "p_meda"
        data = moderate[moderate["eps"]==eps]
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        surf=ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        ax.set_xlabel(r'$\gamma$', fontsize=40, labelpad=30)
        ax.set_ylabel(r'$p_m$', fontsize=50, labelpad=30)
        ax.tick_params(axis='x', which='major', labelrotation=45)
        ax.tick_params(axis='y', which='major', pad=10)
        ax.tick_params(axis='z', which='major', pad=10)  
        plt.savefig(f'plots/moderate/{measure}/moderate_{measure}_e{eps}_surface.png', bbox_inches="tight")
        plt.close()      

### spaghetti

In [49]:
import sys
sys.path.append("/../Lib/")
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")


n = 100
graph = nx.complete_graph(n)
max_it = 1000000

for mo in [[0.5]]:
    for pm in [0.0, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                print(f"doing spaghetti_moderate_pm{pm}_e{e}_g{g}")
                model = op.AlgorithmicBiasMediaModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("gamma_media", g)
                config.add_model_parameter("k", len(mo))
                config.add_model_parameter("p", pm)
                model.set_initial_status(config)
                model.set_media_opinions(mo)
                # # Simulation execution
                iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/moderate/spaghetti_moderate_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                plt.close()

  0%|                                                                          | 17/1000000 [00:00<1:42:32, 162.54it/s]

doing spaghetti_moderate_pm0.0_e0.2_g0.0


  0%|                                                                          | 17/1000000 [00:00<1:39:11, 168.02it/s]

doing spaghetti_moderate_pm0.0_e0.2_g0.5


  0%|                                                                          | 33/1000000 [00:00<1:41:11, 164.69it/s]

...

  0%|                                                                          | 42/1000000 [00:00<1:22:16, 202.55it/s]

doing spaghetti_moderate_pm0.5_e0.5_g1.0


  0%|                                                                          | 20/1000000 [00:00<1:25:14, 195.51it/s]

doing spaghetti_moderate_pm0.5_e0.5_g1.5


  0%|                                                                        | 1036/1000000 [00:05<1:33:08, 178.75it/s]


### Per presentazione

for e in [0.2, 0.3, 0.4, 0.5]:
    for g in [0.0, 1.0, 1.5]:
        for pm in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            try:
                fig, ax = plt.subplots(figsize=(5, 3))
                mo = '0.5'
                name = f'for_spaghetti/for_spaghetti media mo[{mo}] p{pm} e{e} g{g} gm{g} mi1000000'
                iterations = from_res_to_iterations(name, 1)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/moderate/spaghetti_moderate_e{e}_g{g}_pm{pm}.png", bbox_inches="tight")
                plt.close()
            except FileNotFoundError:
                plt.close()
                continue

# Extremist media - propaganda

### Heatmaps


In [40]:
sns.set(font_scale=1.0)
for eps in sorted(extremist.eps.unique()):
    for measure in measures:
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = extremist[extremist["eps"]==eps]
        df = data.pivot(x, y, measure)
        if measure == "avg_niter":
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".0f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(extremist[measure]), vmax=max(extremist[measure]))
        else:
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(extremist[measure]), vmax=max(extremist[measure]))
        ax.set_xlabel(r'$\gamma$', fontsize=15)
        ax.set_ylabel(r'p_m', fontsize=15)
        ax.set_title(r"$\epsilon$={}".format(eps), fontsize=20)
        plt.savefig(f"plots/extremist/{measure}/extremist_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
        # relativa colorbar 
        fig, ax = plt.subplots(figsize=(7, 1), dpi=600)
        fig.subplots_adjust(bottom=0.5)
        cmap = heatmap_cmap
        norm = mpl.colors.Normalize(vmin=min(extremist[measure]), vmax=max(extremist[measure]))
        cb = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=heatmap_cmap),
                    cax=ax, orientation='horizontal')
        cb.outline.set_visible(False)
        plt.savefig(f"plots/extremist/colorbar_extremist_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()

### Spaghetti

In [48]:
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")


n = 100
graph = nx.complete_graph(n)
max_it = 1000000

for mo in [[0.0]]:
    for pm in [0.0, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                print(f"doing spaghetti_extremist_pm{pm}_e{e}_g{g}")
                model = op.AlgorithmicBiasMediaModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("gamma_media", g)
                config.add_model_parameter("k", len(mo))
                config.add_model_parameter("p", pm)
                model.set_initial_status(config)
                model.set_media_opinions(mo)
                # # Simulation execution
                iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/extremist/spaghetti_extremist_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                plt.close()

  0%|                                                                          | 28/1000000 [00:00<1:00:07, 277.17it/s]

doing spaghetti_extremist_pm0.0_e0.2_g0.0


  0%|                                                                          | 19/1000000 [00:00<1:31:01, 183.11it/s]

doing spaghetti_extremist_pm0.0_e0.2_g0.5


  0%|                                                                            | 29/1000000 [00:00<59:27, 280.30it/s]

...

  0%|                                                                          | 15/1000000 [00:00<1:55:13, 144.64it/s]

doing spaghetti_extremist_pm0.5_e0.5_g1.0


  0%|                                                                          | 30/1000000 [00:00<2:05:31, 132.77it/s]

doing spaghetti_extremist_pm0.5_e0.5_g1.5


  1%|▍                                                                       | 5591/1000000 [00:55<2:45:14, 100.30it/s]


### Surface

In [41]:
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
for measure in measures:
    for eps in sorted(extremist.eps.unique()):
        x = "gam"
        y = "p_meda"
        data = extremist[extremist["eps"]==eps]
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        surf=ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        ax.set_xlabel(r'$\gamma$', fontsize=40, labelpad=30)
        ax.set_ylabel(r'$p_m$', fontsize=50, labelpad=30)
        ax.tick_params(axis='x', which='major', labelrotation=45)
        ax.tick_params(axis='y', which='major', pad=10)
        ax.tick_params(axis='z', which='major', pad=10)  
        plt.savefig(f'plots/extremist/{measure}/extremist_{measure}_e{eps}_surface.png', bbox_inches="tight")
        plt.close()      

# Polarised media landscape

### Heatmap

In [42]:
sns.set(font_scale=1.0)
for eps in sorted(polarised.eps.unique()):
    i=0
    for measure in measures:
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = polarised[polarised["eps"]==eps]
        df = data.pivot(x, y, measure)
        if measure == "avg_niter":
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".0f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(polarised[measure]), vmax=max(polarised[measure]))
        else:
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(polarised[measure]), vmax=max(polarised[measure]))
        ax.set_xlabel(r'$\gamma$', fontsize=15)
        ax.set_ylabel(r'p_m', fontsize=15)
        ax.set_title(r"$\epsilon$={}".format(eps), fontsize=20)
        plt.savefig(f"plots/polarised/{measure}/polarised_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
        # relativa colorbar 
        fig, ax = plt.subplots(figsize=(7, 1), dpi=600)
        fig.subplots_adjust(bottom=0.5)
        cmap = heatmap_cmap
        norm = mpl.colors.Normalize(vmin=min(extremist[measure]), vmax=max(extremist[measure]))
        cb = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=heatmap_cmap),
                    cax=ax, orientation='horizontal')
        cb.outline.set_visible(False)
        plt.savefig(f"plots/polarised/colorbar_polarised_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()

### Surface

In [43]:
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
for measure in measures:
    for eps in sorted(polarised.eps.unique()):
        x = "gam"
        y = "p_meda"
        data = polarised[polarised["eps"]==eps]
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        surf=ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        ax.set_xlabel(r'$\gamma$', fontsize=40, labelpad=30)
        ax.set_ylabel(r'$p_m$', fontsize=50, labelpad=30)
        ax.tick_params(axis='x', which='major', labelrotation=45)
        ax.tick_params(axis='y', which='major', pad=10)
        ax.tick_params(axis='z', which='major', pad=10)  
        plt.savefig(f'plots/polarised/{measure}/polarised_{measure}_e{eps}_surface.png', bbox_inches="tight")
        plt.close()      

### Spaghetti

In [47]:
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")


n = 100
graph = nx.complete_graph(n)
max_it = 1000000

for mo in [[0.05, 0.95]]:
    for pm in [0.0, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                print(f"doing spaghetti_polarised_pm{pm}_e{e}_g{g}")
                model = op.AlgorithmicBiasMediaModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("gamma_media", g)
                config.add_model_parameter("k", len(mo))
                config.add_model_parameter("p", pm)
                model.set_initial_status(config)
                model.set_media_opinions(mo)
                # # Simulation execution
                iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/polarised/spaghetti_polarised_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                plt.close()

  0%|                                                                            | 60/1000000 [00:00<58:27, 285.06it/s]

doing spaghetti_polarised_pm0.0_e0.2_g0.0


  0%|                                                                          | 54/1000000 [00:00<1:05:28, 254.51it/s]

doing spaghetti_polarised_pm0.0_e0.2_g0.5


  0%|                                                                          | 54/1000000 [00:00<1:02:08, 268.19it/s]

...

  0%|                                                                          | 20/1000000 [00:00<1:24:46, 196.60it/s]

doing spaghetti_polarised_pm0.5_e0.5_g1.0


  0%|                                                                          | 38/1000000 [00:00<1:36:30, 172.68it/s]

doing spaghetti_polarised_pm0.5_e0.5_g1.5


  0%|                                                                        | 1040/1000000 [00:06<1:39:10, 167.86it/s]


# Balanced media landscape

In [44]:
sns.set(font_scale=1.0)
for eps in sorted(balanced.eps.unique()):
    i=0
    for measure in measures:
        fig, ax = plt.subplots(figsize=(5, 5), dpi=600)
        x = "p_media"
        y = "gam"
        data = balanced[balanced["eps"]==eps]
        df = data.pivot(x, y, measure)
        if measure == "avg_niter":
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".0f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(balanced[measure]), vmax=max(balanced[measure]))
        else:
            a=sns.heatmap(df, cbar=True, cbar_kws = dict(use_gridspec=False,location="top"), fmt=".2f", annot=True, ax = ax, cmap=heatmap_cmap, vmin=min(balanced[measure]), vmax=max(balanced[measure]))
        ax.set_xlabel(r'$\gamma$', fontsize=15)
        ax.set_ylabel(r'p_m', fontsize=15)
        ax.set_title(r"$\epsilon$={}".format(eps), fontsize=20)
        plt.savefig(f"plots/balanced/{measure}/balanced_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()
        # relativa colorbar 
        fig, ax = plt.subplots(figsize=(7, 1), dpi=600)
        fig.subplots_adjust(bottom=0.5)

        cmap = heatmap_cmap
        norm = mpl.colors.Normalize(vmin=min(extremist[measure]), vmax=max(extremist[measure]))
        cb = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=heatmap_cmap),
                    cax=ax, orientation='horizontal')
        cb.outline.set_visible(False)
        plt.savefig(f"plots/balanced/colorbar_balanced_{measure}_e{eps}_heatmap.png", bbox_inches="tight")
        plt.close()

In [45]:
sns.set(font_scale=1.0)
sns.set_style("whitegrid")
for measure in measures:
    for eps in sorted(balanced.eps.unique()):
        x = "gam"
        y = "p_meda"
        data = balanced[balanced["eps"]==eps]
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        surf=ax.plot_trisurf(data['gam'], data['p_media'], data[measure], cmap=heatmap_cmap, linewidth=0.0, alpha=0.8)
        ax.set_xlabel(r'$\gamma$', fontsize=40, labelpad=30)
        ax.set_ylabel(r'$p_m$', fontsize=50, labelpad=30)
        ax.tick_params(axis='x', which='major', labelrotation=45)
        ax.tick_params(axis='y', which='major', pad=10)
        ax.tick_params(axis='z', which='major', pad=10)  
        plt.savefig(f'plots/balanced/{measure}/balanced_{measure}_e{eps}_surface.png', bbox_inches="tight")
        plt.close()      

In [ ]:
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")


n = 100
graph = nx.complete_graph(n)
max_it = 1000000

for mo in [[0.05, 0.5, 0.95]]:
    for pm in [0.0, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                print(f"doing spaghetti_balanced_pm{pm}_e{e}_g{g}")
                model = op.AlgorithmicBiasMediaModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("gamma_media", g)
                config.add_model_parameter("k", len(mo))
                config.add_model_parameter("p", pm)
                model.set_initial_status(config)
                model.set_media_opinions(mo)
                # # Simulation execution
                iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/balanced/spaghetti_balanced_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                plt.close()

In [ ]:
import sys
sys.path.append("/../Lib/")
import sys
sys.path.append("/../Lib/")
sys.path.append("/../Lib/netdspatch_local/")
sys.path.append("/../Lib/ndlib_local/")

import json
import os
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")

n = 100
graph = nx.complete_graph(n)
max_it = 1000000
for mo in [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]:
    for pm in [0.1, 0.2, 0.3, 0.4]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0]:
                print(f"doing {name}")
                model = op.AlgorithmicBiasMediaModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("gamma_media", g)
                config.add_model_parameter("k", len(mo))
                config.add_model_parameter("p", pm)
                model.set_initial_status(config)
                model.set_media_opinions(mo)
                # # Simulation execution
                iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True, drop_evolution=False)
                fig, ax = plt.subplots(figsize=(7, 5), dpi=600)
                evolution(iterations, spaghetti_hex_list, fig=fig, ax=ax)
                plt.title(r'$\epsilon$='+f'{e},'+r'$\gamma$='+f'{g},'+r'$p_m$='+f'{pm}')
                plt.savefig(f"plots/deffuant/spaghetti_mo{mo}_pm{pm}_e{e}_g{g}.png", bbox_inches="tight")
                plt.close()